In [1]:
import dask
from dask.distributed import Client, progress, LocalCluster, performance_report
import numpy as np
import pathlib
import pandas as pd
from rechunker import rechunk
import shutil
import fsspec
import xarray as xr
from datetime import datetime

In [2]:
n_workers = 10
scheduler = LocalCluster(n_workers=n_workers, threads_per_worker=2, processes=False)
client = Client(scheduler)
client.dashboard_link

C:\Users\dsmith\Anaconda3\envs\hms_kube\lib\site-packages\distributed\node.py:181: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 53897 instead
  warnings.warn(


'http://10.104.119.178:53897/status'

In [3]:
nwm_url = 's3://noaa-nwm-retrospective-2-1-zarr-pds/chrtout.zarr'
start_date = datetime.fromisoformat("2000-01-01")
end_date = datetime.fromisoformat("2010-12-31")
variables = ["streamflow", "velocity"]
# comids = [2043493, 20873280, 7590453]
comids = [2043493]

start_string = f"{start_date.year}-{start_date.month}-{start_date.day}"
end_string = f"{end_date.year}-{end_date.month}-{end_date.day}"
time_steps = int((end_date - start_date).total_seconds() / 3600)
time_steps

96408

In [4]:
%%time
ds = xr.open_zarr(fsspec.get_mapper(nwm_url, anon=True), consolidated=True, chunks=None)
ds = ds.chunk({"feature_id": 1000, "time": int(time_steps/2)})
# ds = ds.unify_chunks()
# ds.info

CPU times: total: 9.92 s
Wall time: 44 s


In [5]:
ds = ds.drop('crs')
# ds["feature_id"].to_pandas().to_csv('waterbody-comids.csv', index=False)

In [6]:
%%time
ds_data = ds[variables].sel(feature_id=comids).sel(time=slice(start_string, end_string)).load(optimize_graph=False, traverse=False)
ds_data

distributed.worker - WARNING - Compute Failed
Function:  subgraph_callable-bff70d6f-aece-44e3-b43e-171b4feb
args:      (ImplicitToExplicitIndexingAdapter(array=MemoryCachedArray(array=CopyOnWriteArray(array=LazilyIndexedArray(array=_ElementwiseFunctionArray(_ElementwiseFunctionArray(LazilyIndexedArray(array=<xarray.backends.zarr.ZarrArrayWrapper object at 0x000001FBE8F31D00>, key=BasicIndexer((slice(None, None, None), slice(None, None, None)))), func=functools.partial(<function _apply_mask at 0x000001FBE5C26940>, encoded_fill_values={-999900}, decoded_fill_value=nan, dtype=dtype('float64')), dtype=dtype('float64')), func=functools.partial(<function _scale_offset_decoding at 0x000001FBE5C269D0>, scale_factor=0.009999999776482582, add_offset=0.0, dtype=<class 'numpy.float64'>), dtype=dtype('float64')), key=BasicIndexer((slice(None, None, None), slice(None, None, None))))))), (slice(96408, 192816, None), slice(180000, 190000, None)))
kwargs:    {}
Exception: "ClientPayloadError('Response 

ClientPayloadError: Response payload is not completed

In [7]:
%%time
with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ds_data = ds[variables].sel(feature_id=comids).sel(time=slice(start_string, end_string)).load(optimize_graph=False, traverse=False)
ds_data

distributed.worker - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker.html#memtrim for more information. -- Unmanaged memory: 8.98 GiB -- Worker memory limit: 12.78 GiB
distributed.worker - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker.html#memtrim for more information. -- Unmanaged memory: 8.99 GiB -- Worker memory limit: 12.78 GiB
distributed.worker - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker.html#memtrim for more information. -- Unmanaged memory: 9.00 GiB -- Worker memory limit: 12.78 GiB
distributed.worker - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; s

CPU times: total: 7min 13s
Wall time: 33min 53s


<xarray.Dataset>
Dimensions:     (time: 96432, feature_id: 1)
Coordinates:
    elevation   (feature_id) float32 347.1
  * feature_id  (feature_id) int32 2043493
    gage_id     (feature_id) |S15 b'               '
    latitude    (feature_id) float32 34.43
    longitude   (feature_id) float32 -83.74
    order       (feature_id) int32 2
  * time        (time) datetime64[ns] 2000-01-01 ... 2010-12-31T23:00:00
Data variables:
    streamflow  (time, feature_id) float64 0.44 0.44 0.44 ... 0.45 0.45 0.45
    velocity    (time, feature_id) float64 0.39 0.39 0.39 ... 0.39 0.39 0.39
Attributes:
    TITLE:                OUTPUT FROM WRF-Hydro v5.2.0-beta2
    code_version:         v5.2.0-beta2
    featureType:          timeSeries
    model_configuration:  retrospective
    proj4:                +proj=lcc +units=m +a=6370000.0 +b=6370000.0 +lat_1...

In [ ]:
%%time
ds_data = ds[variables].sel(feature_id=comids).sel(time=slice(start_string, end_string)).load(optimize_graph=True, traverse=False)
ds_data

distributed.worker - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker.html#memtrim for more information. -- Unmanaged memory: 8.95 GiB -- Worker memory limit: 12.78 GiB
distributed.worker - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker.html#memtrim for more information. -- Unmanaged memory: 8.97 GiB -- Worker memory limit: 12.78 GiB
distributed.worker - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker.html#memtrim for more information. -- Unmanaged memory: 8.94 GiB -- Worker memory limit: 12.78 GiB
distributed.worker - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; s

In [ ]:
%%time
with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ds_data = ds[variables].sel(feature_id=comids).sel(time=slice(start_string, end_string)).load(optimize_graph=True, traverse=False)
ds_data